In [1]:
import ross as rs
import numpy as np
import plotly.graph_objects as go

Q_ = rs.Q_
rs.__version__

'1.5.0'

Just claculate the first rotor, need to have this too take the matrice, that we going to use

In [2]:
material_exo = rs.Material(name="mat_exo", rho=7850, E=2.05 * 10**11, Poisson=0.3)


Low_pressure_elements = []
low_pressure_seal_elements = []
low_pressure_elements = []

for i in range(8) : 
    Low_pressure_elements.append(rs.ShaftElement(L = 0.1, material=material_exo, n=i, idl=0, odl=0.05))

low_pressure_elements.append(
    rs.DiskElement.from_geometry(n=1, material=material_exo, width=0.02, i_d=0.05, o_d=0.257)
)

low_pressure_elements.append(
    rs.DiskElement.from_geometry(n=7, material=material_exo, width=0.02, i_d=0.05, o_d=0.3)
)

low_pressure_seal_elements.append(rs.BearingElement(n=0, kxx=3e7, kyy=3e7, cxx=0, cyy=0))
low_pressure_seal_elements.append(rs.BearingElement(n=8, kxx=3e7, kyy=3e7, cxx=0, cyy=0))
low_pressure_seal_elements.append(rs.BearingElement(n=6, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0))

rotor_lp = rs.Rotor(
    shaft_elements=Low_pressure_elements,
    bearing_elements=low_pressure_seal_elements,
    disk_elements=low_pressure_elements,
)

rotor_lp.plot_rotor()

In [3]:
samples = 40
max_spin = 20000/60 * 2 * np.pi
speed_range = np.linspace(0, max_spin, samples)
campbell = rotor_lp.run_campbell(speed_range, frequency_type="wn", frequencies=7)
fig = campbell.plot(frequency_units="rad/s", width=600, height=600)
# for i in fig.data:
#     try:
#         i["y"] = i["y"] / 60  * 2 * np.pi # pass in hz   
#     except:
#         pass
fig.update_yaxes(title="Natural Frequencies (Hz)", range=[0, 90])
fig.update_xaxes(title="Rotor Spin Speed (rpm)", range=[0, max_spin * 60 / (2 * np.pi)])

In [4]:
hight_pressure_elements = []
hight_pressure_seal_elements = []
hight_pressure_disk_elements = []

for i in range(4) : 
    hight_pressure_elements.append(rs.ShaftElement(L = 0.1, material=material_exo, n=i, idl=0.07, odl=0.08))

hight_pressure_disk_elements.append(
    rs.DiskElement.from_geometry(n=2, material=material_exo, width=0.015, i_d=0.08, o_d=0.038)
)

hight_pressure_seal_elements.append(rs.BearingElement(n=0, kxx=2.5e7, kyy=2.5e7, cxx=0, cyy=0))
hight_pressure_seal_elements.append(rs.BearingElement(n=4, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0))
rotor_hp = rs.Rotor(
    shaft_elements=hight_pressure_elements,
    bearing_elements=hight_pressure_seal_elements,
    disk_elements=hight_pressure_disk_elements,
)

rotor_hp.plot_rotor()


In [5]:
campbell = rotor_hp.run_campbell(speed_range, frequency_type="wn", frequencies=7)
fig = campbell.plot(frequency_units="rad/s", width=600, height=600)
fig

In [6]:
modal = rotor_hp.run_modal(53.14, 7)
modal.wd[:1]

array([3637.56286723])

Need to create one birotor with two shft two extract the therm K and M, which didn't depend on the gyroscopique effect. And so on the RPM

In [7]:

axial_shaft = [rs.ShaftElement(0.1, 0, 0.05, material=material_exo) for _ in range(8)]


coaxial_shaft = [rs.ShaftElement(0.1, 0.07, 0.08, material=material_exo) for _ in range(4)]


disks = [
    rs.DiskElement.from_geometry(n=1, material=material_exo, width=0.02, i_d=0.05, o_d=0.257),
    rs.DiskElement.from_geometry(n=7, material=material_exo, width=0.02, i_d=0.05, o_d=0.3),
    rs.DiskElement.from_geometry(n=11, material=material_exo, width=0.015, i_d=0.08, o_d=0.038)
]

bearings = [
    rs.BearingElement(0, kxx=3*10**7, kyy=3*10**7, cxx=0),
    rs.BearingElement(8, kxx=3*10**7, kyy=3*10**7, cxx=0),
    rs.BearingElement(9, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0),
    rs.BearingElement(6, n_link=13, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0)
]
""
# Création du rotor coaxial
shaft = [axial_shaft, coaxial_shaft]
rotor_coaxial = rs.CoAxialRotor(shaft, disks, bearings)
rotor_coaxial.plot_rotor()

In [8]:
# modal = rotor_coaxial.run_modal(4,)
# fig = modal.plot_mode_3d(0, frequency_units="rpm", width=600, height=600)
# fig.layout.title.text = "FIRST CRITICAL SPEED" + fig.layout.title.text
# # fig.show()

In [20]:
modal = rotor_coaxial.run_modal(speed=Q_(0, "RPM"))
fig = modal.plot_mode_3d(1, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "SECOND CRITICAL SPEED" + fig.layout.title.text
fig.show()

Extract the diverse matrix

In [10]:
G_lp = rotor_lp.G()
print("Gyrioscopique Low pressure \t",G_lp.shape)
G_hp = rotor_hp.G() * 1.5
print("Gyrioscopique High pressure \t",G_hp.shape)

# campbell = rotor_coaxial.run_campbell(speed_range=Q_(list(range(0, 4500, 50)), "RPM"))


Gyrioscopique Low pressure 	 (36, 36)
Gyrioscopique High pressure 	 (20, 20)


In [11]:
G1 = np.hstack((G_lp, np.zeros((G_lp.shape[0], rotor_coaxial.ndof - G_lp.shape[1]))))
G2 = np.hstack((np.zeros((G_hp.shape[0],rotor_coaxial.ndof - G_hp.shape[1])), G_hp))

G = np.vstack((G1, G2))

In [12]:
import other_fct as fct
max_spin = 20000/60 * 2 * np.pi
samples = 40
speed_range = np.linspace(0, max_spin, samples)
fct.run_campbell(rotor_coaxial, speed_range, frequencies = 7, Gyro=G) 

damping_ratio =  [-4.14839748e-15  3.46510849e-15 -2.24907468e-13 -2.68602732e-14
  5.69510742e-15 -5.92921856e-14  5.12014387e-11]
damping_ratio =  [ 3.47761506e-14 -3.51767349e-14 -2.05747333e-13  1.66468326e-13
  3.62996345e-12 -3.79396034e-12  4.40971216e-13]
damping_ratio =  [-4.39378426e-14  4.67347226e-14  2.84512668e-12 -2.79406841e-12
 -1.18744639e-12  1.06324730e-12 -6.87410331e-13]
damping_ratio =  [ 5.29431422e-14 -5.46151478e-14  9.60034331e-13 -9.44681169e-13
 -5.20074450e-12  5.01625894e-12  1.13288756e-11]
damping_ratio =  [-1.89085173e-14  3.01606908e-14  1.84292550e-12 -1.81235490e-12
 -2.57271188e-12  2.44400580e-12  2.70152100e-13]
damping_ratio =  [-1.00942124e-14  9.22884081e-15  2.41357682e-12 -2.49944217e-12
  7.13961291e-12 -8.37029994e-12  7.59644638e-12]
damping_ratio =  [-1.05758397e-14  1.94329141e-14 -4.98693061e-13  5.31294400e-13
  1.42014676e-12 -2.57385530e-12 -3.62201586e-12]
damping_ratio =  [ 1.09312737e-14 -1.49863051e-14  1.66058755e-12 -1.7198326

In [13]:
fct.run_cambell_2_rotor(rotor_lp, rotor_hp, speed_range, frequencies = 7)

damping_ratio =  [-3.45322430e-16  1.07266152e-15  4.64887539e-15]
damping_ratio =  [ 1.16894277e-13 -8.67525410e-14  1.26024285e-13]
damping_ratio =  [-2.39655703e-14  2.47145484e-14 -1.76019789e-13]
damping_ratio =  [ 8.91752486e-14 -9.14405249e-14  3.44541414e-13]
damping_ratio =  [-6.18496476e-14  5.89145498e-14  4.64689275e-13]
damping_ratio =  [ 3.59035908e-12 -3.62980416e-12  2.23223261e-12]
damping_ratio =  [-1.60820857e-15  7.54629544e-15  2.14971146e-12]
damping_ratio =  [ 1.22585556e-10 -1.22174633e-10  2.10213933e-12]
damping_ratio =  [ 3.29229108e-14 -2.88317740e-14  4.53608169e-13]
damping_ratio =  [ 4.07901730e-13 -4.46017809e-13  1.49909957e-14]
damping_ratio =  [-1.23929551e-13  1.27105552e-13  4.69705372e-13]
damping_ratio =  [ 2.23685143e-12 -2.25345108e-12 -1.44106092e-13]
damping_ratio =  [-7.65842102e-14  8.46922349e-14  1.16827753e-12]
damping_ratio =  [-6.73815746e-13  6.62600060e-13  2.65535828e-18]
damping_ratio =  [ 1.48689710e-14 -2.19419561e-14  1.34829134e

In [14]:
fct.run_campbell(rotor_lp, speed_range, frequencies = 7)

damping_ratio =  [-2.57057937e-14  3.69679463e-14 -7.36408530e-16 -1.26799865e-13
 -1.53245828e-14  3.45355439e-14 -2.29324826e-15]
damping_ratio =  [ 2.00322073e-14 -1.32136083e-14  6.33133486e-14 -6.20821816e-14
  1.14161037e-13 -1.11172453e-13 -7.36523619e-13]
damping_ratio =  [ 1.11814682e-14 -1.21056511e-14  9.66626550e-13 -9.18859184e-13
 -3.68019004e-13  4.78614279e-13 -5.24817941e-13]
damping_ratio =  [ 3.55317586e-14 -3.76664111e-14  2.43517568e-12 -2.45871350e-12
  2.29439608e-13 -2.39548904e-13 -2.80744943e-13]
damping_ratio =  [-2.33371264e-14  2.54742918e-14 -3.91796135e-13  3.88610815e-13
 -1.21026397e-12  1.46367585e-12 -2.39149387e-12]
damping_ratio =  [-2.22177232e-14  3.12433523e-14  5.48752925e-13 -5.74472979e-13
 -3.42179599e-13  3.27706061e-13  1.69652002e-12]
damping_ratio =  [ 7.40117378e-14 -7.58167405e-14  4.80445233e-12 -4.82683537e-12
  4.92892569e-13 -3.93234772e-13 -7.47370562e-13]
damping_ratio =  [ 3.43282278e-14 -2.77689915e-14 -2.66664878e-13  2.1295459

In [17]:
fct.run_campbell(rotor_hp, speed_range, frequencies = 7, Gyro=G_hp*1.5, slope_critic_speed=1.5)

damping_ratio =  [ 4.61288641e-16  2.14315789e-15  5.13801864e-15 -6.54187981e-15
  5.29536799e-14  2.35662646e-15  4.63247147e-14]
damping_ratio =  [-5.26658969e-13  5.01955569e-13 -1.51257027e-14  1.32196921e-13
  4.10374069e-10 -4.12478459e-10 -8.72918962e-10]
damping_ratio =  [-2.80279547e-12  2.81221157e-12  5.36033477e-14  4.56782645e-14
 -5.41906603e-11  5.29894651e-11 -6.72562902e-10]
damping_ratio =  [-1.12153321e-12  1.08460835e-12  3.95291382e-13 -2.08720649e-13
  1.04575171e-10 -1.04937951e-10 -3.01699863e-09]
damping_ratio =  [ 9.65820798e-13 -9.16662991e-13  2.92836584e-13 -1.40033298e-14
 -6.13542949e-11  6.25645313e-11  2.21390934e-09]
damping_ratio =  [ 2.19056283e-13 -1.72959374e-13  2.04292183e-13 -4.48131155e-13
  1.90752789e-10 -1.95365828e-10 -4.10494103e-10]
damping_ratio =  [-1.03656765e-11  1.04169788e-11 -3.14202584e-13  4.01128337e-13
  4.92213586e-11 -5.15917763e-11 -9.73782358e-10]
damping_ratio =  [-4.50082675e-12  4.57349546e-12 -2.45511086e-13  2.1836712

In [16]:
fct.run_damping_mode(rotor_lp, rotor_hp, speed_range, frequencies = 7)

damping_ratio =  [-2.08637425e-14  2.36695616e-14  3.24236308e-15]
damping_ratio =  [-7.45682825e-15  8.05115627e-15 -1.30333495e-14]
damping_ratio =  [ 2.79574279e-14 -3.18734068e-14  8.96193206e-14]
damping_ratio =  [-2.92410431e-13  2.95535929e-13 -6.64223148e-14]
damping_ratio =  [-4.34201739e-14  5.01447447e-14  1.30833575e-12]
damping_ratio =  [-1.52928870e-12  1.54633764e-12 -9.42371163e-13]
damping_ratio =  [-1.09972674e-14  1.35555755e-14 -1.21106216e-12]
damping_ratio =  [ 1.75648739e-11 -1.76146114e-11  1.21049668e-13]
damping_ratio =  [ 2.01227148e-14 -2.20370353e-14 -9.29673912e-13]
damping_ratio =  [ 2.17025938e-11 -2.17807761e-11 -2.97991501e-13]
damping_ratio =  [-2.69175041e-14  2.44374594e-14  2.99486542e-14]
damping_ratio =  [-3.20568215e-12  3.22935532e-12 -6.72513555e-14]
damping_ratio =  [ 3.09996924e-14 -3.61154282e-14  1.29371941e-13]
damping_ratio =  [ 2.26685459e-12 -2.42777066e-12 -5.10618050e-13]
damping_ratio =  [ 6.58805328e-14 -7.35330847e-14  1.49592587e